In [25]:
import Pkg; Pkg.instantiate()

┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing /Users/shufanzhang/Documents/coderepos/Julia_DataScience/Manifest.toml:0
   Installed Calculus ────────────────── v0.5.1
   Installed x265_jll ────────────────── v3.0.0+3
   Installed JpegTurbo_jll ───────────── v2.1.0+0
   Installed ScikitLearnBase ─────────── v0.5.0
   Installed ImageIO ─────────────────── v0.4.1
   Installed AxisArrays ──────────────── v0.4.3
   Installed MutableArithmetics ──────── v0.2.19
   Installed Calculus ────────────────── v0.5.1
   Installed x265_jll ────────────────── v3.0.0+3
   Installed JpegTurbo_jll ───────────── v2.1.0+0
   Installed ScikitLearnBase ─────────── v0.5.0
   Installed ImageIO ─────────────────── v0.4.1
   Installed AxisArrays ──────────────── v0.4.3
   Installed MutableArithmetics ──────── v0.2.19
   Installed DoubleFloats ────────────── v1.1.22
   Installed DoubleFloats ──────

Pkg.Resolve.ResolverError: Unsatisfiable requirements detected for package [38;5;13mBinaryProvider [b99e7846][39m:
 [38;5;13mBinaryProvider [b99e7846][39m log:
 ├─possible versions are: [38;5;13m0.4.0 - 0.5.10[39m or uninstalled
 ├─restricted to versions [38;5;13m0.3 - 0.5[39m by [38;5;11mproject [e57c8627][39m, leaving only versions: [38;5;13m0.4.0 - 0.5.10[39m
 │ └─[38;5;11mproject [e57c8627][39m log:
 │   ├─possible versions are: [38;5;11m0.0.0[39m or uninstalled
 │   └─[38;5;11mproject [e57c8627][39m is fixed to version [38;5;11m0.0.0[39m
 └─restricted by julia compatibility requirements to versions: uninstalled — no versions left

In [ ]:
import Pkg; Pkg.update()

## Neural Networks
In this notebook, we will walk through one main neural nets example. And that is, classifying the infamous MNIST dataset. **If you have no experience with neural nets prior to this notebook, I recommend doing a quick search for an "intro to neural nets"**, there are multiple tutorials/blog posts out there and you can choose the one that works for you.

Here, we will use the `Flux` package, but if you want to look at other packages I encourage you to look at `Knet.jl` and `TensorFlow.jl`.

In [26]:
using Flux, Flux.Data.MNIST
using Flux: onehotbatch, argmax, crossentropy, throttle
using Base.Iterators: repeated
using Images

ERROR: LoadError: UndefVarError: `@_pure_meta` not defined in `Base`
Stacktrace:
 [1] top-level scope
   @ stdin:6
in expression starting at /Users/shufanzhang/.julia/packages/Compat/GVQpP/src/Compat.jl:238
in expression starting at /Users/shufanzhang/.julia/packages/Compat/GVQpP/src/Compat.jl:1
in expression starting at stdin:6
UndefVarError: `@_pure_meta` not defined in `Base`
Stacktrace:
 [1] top-level scope
   @ stdin:6
in expression starting at /Users/shufanzhang/.julia/packages/Compat/GVQpP/src/Compat.jl:238
in expression starting at /Users/shufanzhang/.julia/packages/Compat/GVQpP/src/Compat.jl:1
in expression starting at stdin:6
ERROR: LoadError: ERROR: LoadError: Failed to precompile Compat [34da2185-b29b-5c13-b0c7-acf172513d20] to "/Users/shufanzhang/.julia/compiled/v1.11/Compat/jl_M4wq9D".
Stacktrace:
 [1] mkpidlock(f::Base.var"#1110#1111"{Base.PkgId}, at::String, pid::Int32; kwopts::@Kwargs{stale_age::Int64, wait::Bool})
   @ FileWatching.Pidfile /opt/homebrew/Cellar/julia/1

ErrorException: Failed to precompile Flux [587475ba-b771-5e3f-ad9e-33799f191a9c] to "/Users/shufanzhang/.julia/compiled/v1.11/Flux/jl_uuDY6E".

Let's take a look at one of the images.

In [27]:
imgs = MNIST.images()
colorview(Gray, imgs[100])

UndefVarError: UndefVarError: `MNIST` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [28]:
typeof(imgs[3])

UndefVarError: UndefVarError: `imgs` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

First, we will transofrm the gray scale values to Float32 types. Here, using Float32 will speedup the neural network substantially withough compromising the quality of the solution.

In [29]:
myFloat32(X) = Float32.(X)
fpt_imgs = myFloat32.(imgs) 

UndefVarError: UndefVarError: `imgs` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [30]:
typeof(fpt_imgs[3])

UndefVarError: UndefVarError: `fpt_imgs` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

We will now create a few helpful functions...

In [31]:
vectorize(x) = x[:]
vectorized_imgs = vectorize.(fpt_imgs);

UndefVarError: UndefVarError: `fpt_imgs` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [32]:
typeof(vectorized_imgs)

UndefVarError: UndefVarError: `vectorized_imgs` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

We will again make use of `...` as the splat operator to concatenate all images into one matrix.

In [33]:
X = hcat(vectorized_imgs...)
size(X)

UndefVarError: UndefVarError: `vectorized_imgs` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Now, every column in `X` is an image of a number. We have `60,000` images. When reshaped into a 28-by-28 matrix, and displayed as an image, can be seen as a handwritten number. Here is an example below.

In [34]:
onefigure = X[:,3]
t1 = reshape(onefigure,28,28)
colorview(Gray,t1)

UndefVarError: UndefVarError: `X` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Next, we will obtain the labels. These are the true labels for the `60,000` images.

In [35]:
labels = MNIST.labels()
labels[1]

UndefVarError: UndefVarError: `MNIST` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

From these labels, we will create a new output column for each image. These columns will be indicator vectors of where the correct label is.

For example if the figure corresponding to column `X[:,i]` is a `3`, the `i'th` column in this new matrix `Y` is `[0 0 0 1 0 0 0 0 0 0]`. (It is the entry number 4 because entry 1 corresponds to the digit 0, so the counting starts from zero). The `onehotbatch` function allows us to create this easily.

In [36]:
Y = onehotbatch(labels, 0:9)

UndefVarError: UndefVarError: `onehotbatch` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

And now we will actually build our neural network. We will use two layers. The hidden layer will have 32 nodes, and the output layer will have 10 nodes. i.e. we will go from: `28*28 => 32 => 10`.

In [37]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax)

UndefVarError: UndefVarError: `Chain` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

What does `m`, the neural network mean here? 

If you've worked with neural networks before you know that the solution is often not found by just one pass on the neural network. One pass happens, and a solution is generated at the output layer, then this solution is compared to the ground truth solution we already have (the columns from `Y`), and the network goes back and adjusts its weights and parameters and then try again. Here, since `m` is not "trained" yet, one pass of `m` on a figure generates the following (not-so-great) answer. We will see later how this changes after training.

In [38]:
m(onefigure)

UndefVarError: UndefVarError: `m` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

To run our neural network, we need a loss function and an accuracy function. The accuracy function is used to compare the output result from the output layer in the neural network to the groundtruth result. The loss function is used to evaluate the performance of the overall model after new weights have been recalculated at each pass.

In [39]:
loss(x, y) = Flux.crossentropy(m(x), y)
accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))

accuracy (generic function with 1 method)

Finally, we will repeat our data so that we have more samples to pass to the neural network, which means there will be more chances for corrections.

In [40]:
datasetx = repeated((X, Y), 200)
C = collect(datasetx);

UndefVarError: UndefVarError: `repeated` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

We will create a function to display the loss at each step.

In [41]:
evalcb = () -> @show(loss(X, Y))

#13 (generic function with 1 method)

In [42]:
ps = Flux.params(m)

UndefVarError: UndefVarError: `Flux` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Finally, we are ready to train the model, we will use the `Flux.train!` function. Let's take a look at the documentation.

In [43]:
?Flux.train!

Base.Meta.ParseError: ParseError:
# Error @ /Users/shufanzhang/Documents/coderepos/Julia_DataScience/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X43sZmlsZQ==.jl:1:1
?Flux.train!
╙ ── not a unary operator

In [44]:
opt = ADAM()
Flux.train!(loss, ps, datasetx, opt, cb = throttle(evalcb, 10))

UndefVarError: UndefVarError: `ADAM` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

We will now get the test data.

In [45]:
tX = hcat(float.(reshape.(MNIST.images(:test), :))...);
test_image = m(tX[:,1])

UndefVarError: UndefVarError: `MNIST` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [46]:
argmax(test_image) - 1

UndefVarError: UndefVarError: `test_image` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [47]:
t1 = reshape(tX[:,1],28,28)
colorview(Gray, t1)

UndefVarError: UndefVarError: `tX` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

What about the image we tried a few cells earlier and returned the "not-so-great" answer.

In [48]:
onefigure = X[:,2]
m(onefigure)

UndefVarError: UndefVarError: `X` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [49]:
Y[:,2]

UndefVarError: UndefVarError: `Y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# Finally...
After finishing this notebook, you should be able to:
- [ ] prepare data to fit the format to create a neural network using Flux.jl
- [ ] create a neural network with Flux.jl
- [ ] creating an accuracy function and loss function to be passed to train the neural network
- [ ] train the neural network
- [ ] describe a few tips that can help make your nerual network faster or more accurate (such as using Float32 as opposed to Float32)

# 🥳 One cool finding

We ran a trained a neural network on a dataset of of handwritten digits (called the MNIST dataset). At the end, we were able to pass this figure to the neural network and the return result was:

<img src="data/1001.png" width="40">

```
10-element Array{Float32,1}:
 0.00029263002
 1.5993925f-5
 0.0002862561
 0.0035434738
 1.388653f-5
 2.4878627f-5
 6.433018f-7
 0.99414164 ### <= this is the highest number!
 0.000118321994
 0.0015623316
```